# Overview

This notebook illustrates how to use Prefect to schedule and keep track of WarpX simulations running at NERSC.

Prefect is used to track the jobs, and to handle asynchronicity.

In [1]:
from sfapi_client import Client
from sfapi_client.compute import Machine
import prefect
from pathlib import Path

In [6]:
# Create Perlmutter job scripts

def create_warpx_job_script( TOD, z_pos ):
    """
    Creates a job script that runs a WarpX IP2 simulation

    Parameters:
    -----------
    TOD: float
        Value of the TOD in fs^3
    z_pos: float
        Value of the position in microns
    """
    import random
    job_id = random.randint(0, 1000000)
    rundir = "run_%08d" %job_id
    
    script = """#!/bin/bash -l

#SBATCH -t 00:30:00
#SBATCH -N 4
#SBATCH -J WarpX
#SBATCH -A m3239_g
#SBATCH -q regular
#SBATCH --exclusive
#SBATCH --gpu-bind=none
#SBATCH --ntasks-per-node=4
#SBATCH --gpus-per-node=4

cd /pscratch/sd/r/rlehe/ip2/
mkdir %s
cd %s

python ../templates/prepare_simulation.py --TOD=%f --z_pos=%f

export SRUN_CPUS_PER_TASK=16
export EXE="../templates/warpx.2d.MPI.CUDA.DP.PDP.OPMD.EB.QED"
export INPUTS="../templates/template_inputs_2d"

# GPU-aware MPI optimizations
GPU_AWARE_MPI="amrex.use_gpu_aware_mpi=1"
srun --cpu-bind=cores bash -c "
    export CUDA_VISIBLE_DEVICES=\$((3-SLURM_LOCALID));
    ${EXE} ${INPUTS} ${GPU_AWARE_MPI}"
""" %(rundir, rundir, TOD, z_pos)
    return script

<>:18: SyntaxWarning: invalid escape sequence '\$'
<>:18: SyntaxWarning: invalid escape sequence '\$'
/var/folders/_d/m96lcmy90_b9sc5fmhg3d_br0000gn/T/ipykernel_36094/501537247.py:18: SyntaxWarning: invalid escape sequence '\$'
  script = """#!/bin/bash -l


In [7]:
print(create_warpx_job_script(1000, 0))

#!/bin/bash -l

#SBATCH -t 00:30:00
#SBATCH -N 4
#SBATCH -J WarpX
#SBATCH -A m3239_g
#SBATCH -q regular
#SBATCH --exclusive
#SBATCH --gpu-bind=none
#SBATCH --ntasks-per-node=4
#SBATCH --gpus-per-node=4

cd /pscratch/sd/r/rlehe/ip2/
mkdir run_00530934
cd run_00530934

python ../templates/prepare_simulation.py --TOD=1000.000000 --z_pos=0.000000

export SRUN_CPUS_PER_TASK=16
export EXE="../templates/warpx.2d.MPI.CUDA.DP.PDP.OPMD.EB.QED"
export INPUTS="../templates/template_inputs_2d"

# GPU-aware MPI optimizations
GPU_AWARE_MPI="amrex.use_gpu_aware_mpi=1"
srun --cpu-bind=cores bash -c "
    export CUDA_VISIBLE_DEVICES=\$((3-SLURM_LOCALID));
    ${EXE} ${INPUTS} ${GPU_AWARE_MPI}"



In [8]:
# Connect to Perlmutter with the SFAPI
key = Path("./priv_key.pem")
client = Client(key=key)
perlmutter = client.compute(Machine.perlmutter)

@prefect.task
def launch_script_on_perlmutter( script ):
    """
    Launches a script on Perlmutter using the SFAPI,
    and track it with Prefect (using prefect.task)
    """
    # Launch a job with the SFAPI and wait until it finishes
    sfapi_job = perlmutter.submit_job( script )
    # Blocking command that waits for the job to complete 
    sfapi_job.complete()
    return 

@prefect.flow
def run_jobs_serial():
    launch_script_on_perlmutter( create_warpx_job_script(0, 0) )

@prefect.flow
def run_jobs_async():
    async_jobs = [ 
        launch_script_on_perlmutter.submit( random_number_job_script(0) ),
        launch_script_on_perlmutter.submit( random_number_job_script(3) ),
        launch_script_on_perlmutter.submit( random_number_job_script(7) )
    ]
    # Wait for asynchronous jobs to finish
    for r in async_jobs:
        r.result()
    launch_script_on_perlmutter( print_all_script )

In [9]:
# Launch the job and track it with Prefect
run_jobs_serial()

11:46:29.659 | INFO    | prefect.engine - Created flow run 'discreet-uakari' for flow 'run-jobs-serial'

11:46:29.663 | INFO    | prefect.engine - View at https://app.prefect.cloud/account/3f4a5903-b387-41b5-b451-5a982541b3f9/workspace/4f1de454-4acf-4db1-99c0-9a7ba9a6db9c/runs/flow-run/80ef5cb5-2ac4-480e-9462-163e15a39b83

11:46:36.316 | ERROR   | Task run 'launch_script_on_perlmutter-9ce' - Task run failed with exception: SfApiError('sbatch: error: Job request does not match any supported policy.\nsbatch: error: Batch job submission failed: Unspecified error\n') - Retries are exhausted
Traceback (most recent call last):
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/task_engine.py", line 768, in run_context
    yield self
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/task_engine.py", line 1318, in run_task_sync
    engine.call_task_fn(txn)
    ~~~~~~~~~~~~~~~~~~~^^^^^
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/task_engine.py", line 791, in call_task_fn
    result = call_with_parameters(self.task.fn, parameters)
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/utilities/callables.py", line 206, in call_with_parameters
    return fn(*args, **kwargs)
  File "/var/folders/_d/m96lcmy90_b9sc5fmhg3d_br0000gn/T/ipykernel_36094/4158040719.py", line 13, in launch_script_on_perlmutter
    sfapi_job = perlmutter.submit_job( script )
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/sfapi_client/_sync/compute.py", line 34, in wrapper
    return method(self, *args, **kwargs)
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/sfapi_client/_sync/compute.py", line 112, in submit_job
    raise SfApiError(result["error"])
sfapi_client.exceptions.SfApiError: sbatch: error: Job request does not match any supported policy.
sbatch: error: Batch job submission failed: Unspecified error

11:46:36.332 | ERROR   | Task run 'launch_script_on_perlmutter-9ce' - Finished in state Failed('Task run encountered an exception SfApiError: sbatch: error: Job request does not match any supported policy.\nsbatch: error: Batch job submission failed: Unspecified error\n')

11:46:36.336 | ERROR   | Flow run 'discreet-uakari' - Encountered exception during execution: SfApiError('sbatch: error: Job request does not match any supported policy.\nsbatch: error: Batch job submission failed: Unspecified error\n')
Traceback (most recent call last):
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/flow_engine.py", line 655, in run_context
    yield self
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/flow_engine.py", line 699, in run_flow_sync
    engine.call_flow_fn()
    ~~~~~~~~~~~~~~~~~~~^^
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/flow_engine.py", line 678, in call_flow_fn
    result = call_with_parameters(self.flow.fn, self.parameters)
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/utilities/callables.py", line 206, in call_with_parameters
    return fn(*args, **kwargs)
  File "/var/folders/_d/m96lcmy90_b9sc5fmhg3d_br0000gn/T/ipykernel_36094/4158040719.py", line 20, in run_jobs_serial
    launch_script_on_perlmutter( create_warpx_job_script(0, 0) )
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/tasks.py", line 1002, in __call__
    return run_task(
        task=self,
    ...<2 lines>...
        return_type=return_type,
    )
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/task_engine.py", line 1507, in run_task
    return run_task_sync(**kwargs)
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/task_engine.py", line 1320, in run_task_sync
    return engine.state if return_type == "state" else engine.result()
                                                       ~~~~~~~~~~~~~^^
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/task_engine.py", line 475, in result
    raise self._raised
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/task_engine.py", line 768, in run_context
    yield self
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/task_engine.py", line 1318, in run_task_sync
    engine.call_task_fn(txn)
    ~~~~~~~~~~~~~~~~~~~^^^^^
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/task_engine.py", line 791, in call_task_fn
    result = call_with_parameters(self.task.fn, parameters)
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/prefect/utilities/callables.py", line 206, in call_with_parameters
    return fn(*args, **kwargs)
  File "/var/folders/_d/m96lcmy90_b9sc5fmhg3d_br0000gn/T/ipykernel_36094/4158040719.py", line 13, in launch_script_on_perlmutter
    sfapi_job = perlmutter.submit_job( script )
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/sfapi_client/_sync/compute.py", line 34, in wrapper
    return method(self, *args, **kwargs)
  File "/Users/rlehe/miniconda3/envs/sfapi/lib/python3.13/site-packages/sfapi_client/_sync/compute.py", line 112, in submit_job
    raise SfApiError(result["error"])
sfapi_client.exceptions.SfApiError: sbatch: error: Job request does not match any supported policy.
sbatch: error: Batch job submission failed: Unspecified error

11:46:36.530 | ERROR   | Flow run 'discreet-uakari' - Finished in state Failed('Flow run encountered an exception: SfApiError: sbatch: error: Job request does not match any supported policy.\nsbatch: error: Batch job submission failed: Unspecified error\n')

SfApiError: sbatch: error: Job request does not match any supported policy.
sbatch: error: Batch job submission failed: Unspecified error


In [ ]:
# Launch the jobs asynchronously and track them with Prefect
run_jobs_async()